<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: IllinoisGRMHD_headers.h

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the main `IllinoisGRMHD` header file. This module will likely be absorbed by another one by the time we finish documenting the code

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

If using the version of `IllinoisGRMHD` with piecewise polytropic *or* tabulated (coming soon!) EOS support, then the following citation is also required:

* **(Required)** Etienne, Z. B., Werneck, L., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L., *IllinoisGRMHD github repository* (2019). Source Code URL: https://github.com/zachetienne/nrpytutorial/tree/master/IllinoisGRMHD/.

### Dependencies

The files generated in this tutorial notebook depend on the following files:

* `apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.C` \[[**tutorial**](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb)\]
* `IllinoisGRMHD_convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.C` \[[**tutorial**](Tutorial-IllinoisGRMHD__convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.ipynb)\]
* `symmetry__set_gzs_staggered_gfs.C` \[[**tutorial**](Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.ipynb)\]
* `IllinoisGRMHD_EoS_lowlevel_functs.C` \[[**tutorial**](Tutorial-IllinoisGRMHD__EoS_lowlevel_functs.ipynb)\]

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#vars_headers): **Generating `VARS` headers with NRPy+**
    1. [Step 1.a](#load_python_nrpy_modules): *Load necessary Python/NRPy+ modules*
    1. [Step 1.b](#adm_3metric_vars): *The `ADM_3METRIC_VARS.h` file*
    1. [Step 1.c](#conf_metric_facevals_vars): *The `CONF_METRIC_FACEVALS_VARS.h` file*
    1. [Step 1.d](#grmhd_vars): *The `GRMHD_VARS.h` file*
    1. [Step 1.e](#interp_vars): *The `INTERP_VARS.h` file*
    1. [Step 1.f](#smallb_and_conservs_vars): *The `SMALLB_VARS.h` and  `CONSERV_VARS.h` files*
    1. [Step 1.g](#tmunu_vars): *The `TMUNU_VARS.h` files*
1. [Step 2](#igm_headers__h): **`IllinoisGRMHD_headers.h`**
1. [Step 3](#code_validation): **Code validation**
1. [Step 4](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='src_dir'></a>

# Step 0: Source directory creation \[Back to [top](#toc)\]
$$\label{src_dir}$$

We will now use the [cmdline_helper.py NRPy+ module](Tutorial-Tutorial-cmdline_helper.ipynb) to create the source directory within the `IllinoisGRMHD` NRPy+ directory, if it does not exist yet.

In [1]:
# Step 0: Creation of the IllinoisGRMHD source directory
# Step 0a: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0b: Load up cmdline_helper and create the directory
import cmdline_helper as cmd
outdir = os.path.join("..","src")
cmd.mkdir(outdir)

# Step 0c: Set up header file output path
NRPy_headers_dir_path = os.path.join(outdir,"NRPy_generated_headers")
cmd.mkdir(NRPy_headers_dir_path)

<a id='vars_headers'></a>

# Step 1: Generating `VARS` headers with NRPy+ \[Back to [top](#toc)\]
$$\label{vars_headers}$$

We will now use NRPy+ to generate all the `VARS` header files we need in the `IllinoisGRMHD_headers.h` file.

<a id='load_python_nrpy_modules'></a>

## Step 1.a: Load necessary Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{load_python_nrpy_modules}$$

We now load all necessary Python/NRPy+ modules needed to generate the `VARS` header files.

In [2]:
# Imported needed Python modules
import sympy as sp # Python module: symbolic expressions capabilities

# Imported needed NRPy+ modules
import IllinoisGRMHD_output_functions as IGMout # NRPy+ module: IllinoisGRMHD output file functions

<a id='adm_3metric_vars'></a>

## Step 1.b: The `ADM_3METRIC_VARS.h` file \[Back to [top](#toc)\]
$$\label{adm_3metric_vars}$$

We will now generate the `ADM_3METRIC_VARS.h` file, which sets integers so that we can locate specific variables inside the `ADM_3METRIC` array. For example, the lapse variable $\alpha$ is contained in the array element `ADM_3METRIC[0]`, so we define a variable 
```c
const static int ALPHA = 0;
```
which allow us to access the same array element via `ADM_3METRIC[ALPHA]`. We remind the reader that the `ADM_3METRIC` array contains the following quantities (in order):

$$
\left(
\alpha,
\beta^{x},\beta^{y},\beta^{z},
\gamma_{xx},\gamma_{xy},\gamma_{xz},\gamma_{yy},\gamma_{yz},\gamma_{zz},
\gamma^{xx},\gamma^{xy},\gamma^{xz},\gamma^{yy},\gamma^{yz},\gamma^{zz},
\sqrt{\gamma}
\right)\ .
$$

In [3]:
# Step 1.b: Declare basic ADM variables to be used by IllinoisGRMHD
# Step 1.b.i: Set spatial dimension to 3
DIM = 3

# Step 1.b.ii: Set up alpha
gfslist = [["ALPHA"]]
# Step 1.b.iii: Set up beta^{i}
for i in range(DIM):
    gfslist.append(["BETA"+chr(ord('X')+i)])

# Step 1.b.iv: Set up gamma_{ij}
for i in range(DIM):
    for j in range(i,DIM):
        gfslist.append(["GAMMA"+chr(ord('X')+i)+chr(ord('X')+j)])

# Step 1.b.v: Set up gamma^{ij}
for i in range(3):
    for j in range(i,3):
        gfslist.append(["GAMMAUP"+chr(ord('X')+i)+chr(ord('X')+j)])

# Step 1.b.vi: Set up \sqrt{\gamma}
gfslist.append(["SQRTGAMMA"])

# Step 1.b.vii: Set up NUMVARS_FOR_ADM_3METRIC, which is the
#               number of variables in the gfslist array
gfslist.append(["NUMVARS_FOR_ADM_3METRIC"])

# Step 1.b.viii: Output to file
comment = "/* ADM_3METRIC variables */\n"
filename = "ADM_3METRIC_VARS.h"
IGMout.generate_variable_definition_file(gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/ADM_3METRIC_VARS.h


<a id='conf_metric_facevals_vars'></a>

## Step 1.c: The `CONF_METRIC_FACEVALS_VARS.h` file \[Back to [top](#toc)\]
$$\label{conf_metric_facevals_vars}$$

We will now generate the `CONF_METRIC_FACEVALS_VARS.h` file, which sets integers so that we can locate specific variables inside the `CONF_METRIC` and `FACEVALS` arrays. This header file replaces the following piece of code

```c
// The order here MATTERS, as we assume that GAMMAUPXX+1=GAMMAUPYY, etc.
static const int PHI=0,PSI=1,GAMMATILDEXX=2,GAMMATILDEXY=3,GAMMATILDEXZ=4,GAMMATILDEYY=5,GAMMATILDEYZ=6,GAMMATILDEZZ=7,
  LAPM1=8,SHIFTX=9,SHIFTY=10,SHIFTZ=11,GAMMATILDEUPXX=12,GAMMATILDEUPYY=13,GAMMATILDEUPZZ=14,
  NUMVARS_FOR_METRIC_FACEVALS=15; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!

// These are not used for facevals in the reconstruction step, but boy are they useful anyway. 
static const int GAMMAUPXY=15,GAMMAUPXZ=16,GAMMAUPYZ=17,
  NUMVARS_FOR_METRIC=18; //<-- Be _sure_ to set this correctly, or you'll have memory access bugs!
```

with a new addition: we rename *all* conformal metric array quantities by adding a `CM_` string before their names. 

In [4]:
# Start setting up the gridfunction indices with phi and psi
gfslist = [["CM_PHI"],["CM_PSI"]]

# Add the indices for \tilde{\gamma}_{ij}
for i in range(DIM):
    for j in range(i,DIM):
        gfslist.append(["CM_GAMMATILDE"+chr(ord('X')+i)+chr(ord('X')+j)])

# Add alpha, \beta^{i}, and \tilde{\gamma}^{ii}
gfslist.append(["CM_LAPM1"])
for i in range(DIM):
    gfslist.append(["CM_SHIFT"+chr(ord('X')+i)])

for i in range(DIM):
    gfslist.append(["CM_GAMMATILDEUP"+chr(ord('X')+i)+chr(ord('X')+i)])

gfslist.append(["NUMVARS_FOR_CONF_METRIC_FACEVALS"])
othervars =     [["CM_GAMMATILDEUPXY"]]
othervars.append(["CM_GAMMATILDEUPXZ"])
othervars.append(["CM_GAMMATILDEUPYZ"])

# Set up extra code, not supported by the file generating function
extra = "/* Other useful variables */\n"
for j in range(len(othervars)):
    extra += "static const int "+othervars[j][0]
    for k in range(len("NUMVARS_FOR_CONF_METRIC_FACEVALS") - len(othervars[j][0])):
        extra += " "
    extra += " = "+str(j+len(gfslist)-1)+";\n"
extra += "static const int NUMVARS_FOR_CONF_METRIC          = "+str(len(gfslist)+len(othervars)-1)+";\n\n"

# Set up comments
comment = "/* Variables used for face value reconstructions */\n"
# Set up output file name
filename = "CONF_METRIC_FACEVALS_VARS.h"
# Generate variable definition file
IGMout.generate_variable_definition_file(gfslist,filename,comment=comment,extra=extra)

Just generated the file: ../src/NRPy_generated_headers/CONF_METRIC_FACEVALS_VARS.h


<a id='grmhd_vars'></a>

## Step 1.d: The `GRMHD_VARS.h` file \[Back to [top](#toc)\]
$$\label{grmhd_vars}$$

We will now generate the `GRMHD_VARS.h` file, which sets integers so that we can locate specific variables inside the many variations of the primitive arrays (e.g. `prims`, `PRIMS`, `IN_PRIMS`, `OUT_PRIMS_R`, etc). This header file replaces the following piece of code

```c
// The order here MATTERS, and must be consistent with the order in the IN_PRIMS[] array in driver_evaluate_MHD_rhs.C.
static const int RHOB=0,PRESSURE=1,VX=2,VY=3,VZ=4,
  BX_CENTER=5,BY_CENTER=6,BZ_CENTER=7,BX_STAGGER=8,BY_STAGGER=9,BZ_STAGGER=10,
  VXR=11,VYR=12,VZR=13,VXL=14,VYL=15,VZL=16,MAXNUMVARS=17;  //<-- Be _sure_ to define MAXNUMVARS appropriately!
static const int UT=0,UX=1,UY=2,UZ=3;
```

In [5]:
# Add primitives
gfslist =  [["RHOB"],["PRESSURE"],
            ["VX"],["VY"],["VZ"],
            ["BX_CENTER"],["BY_CENTER"],["BZ_CENTER"],
            ["BX_STAGGER"],["BY_STAGGER"],["BZ_STAGGER"],
            ["VXR"],["VYR"],["VZR"],
            ["VXL"],["VYL"],["VZL"],
            ["MAXNUMVARS"]]

# Finally, the 4-velocity
u4list = ["UT","UX","UY","UZ"]

extra = "/* 4-velocity */\n"
for mu in range(4):
    extra += "static const int "+u4list[mu]
    for k in range(len("MAXNUMVARS")-len(u4list[mu])):
        extra += " "
    extra += " = "+str(mu)+";\n"

# Set up comments
comment = "/* GRMHD variables */\n"
# Set up output file name
filename = "GRMHD_VARS.h"
# Generate variable definition file
IGMout.generate_variable_definition_file(gfslist,filename,comment=comment,extra=extra)

Just generated the file: ../src/NRPy_generated_headers/GRMHD_VARS.h


<a id='interp_vars'></a>

## Step 1.e: The `INTERP_VARS.h` file \[Back to [top](#toc)\]
$$\label{interp_vars}$$

We will now generate the `GRMHD_VARS.h` file, which sets integers so that we can locate specific variables inside the many variations of the interpolation array `INTERP_VARS`. This header file replaces the following piece of code

```c
// The "I" suffix denotes interpolation. In other words, these
//    definitions are used for interpolation ONLY. The order here
//    matters as well!
static const int SHIFTXI=0,SHIFTYI=1,SHIFTZI=2,GAMMAUPXXI=3,GAMMAUPXYI=4,GAMMAUPXZI=5,GAMMAUPYYI=6,GAMMAUPYZI=7,GAMMAUPZZI=8,
  PSII=9,LAPM1I=10,A_XI=11,A_YI=12,A_ZI=13,LAPSE_PSI2I=14,LAPSE_OVER_PSI6I=15,MAXNUMINTERP=16;
```

In [6]:
# Set interpolation variables names
gfslist = [["INTERP_SHIFTX"],["INTERP_SHIFTY"],["INTERP_SHIFTZ"],
           ["INTERP_GAMMATILDEUPXX"],["INTERP_GAMMATILDEUPXY"],["INTERP_GAMMATILDEUPXZ"],
           ["INTERP_GAMMATILDEUPYY"],["INTERP_GAMMATILDEUPYZ"],["INTERP_GAMMATILDEUPZZ"],
           ["INTERP_PSI"],["INTERP_LAPM1"],
           ["INTERP_AX"],["INTERP_AY"],["INTERP_AZ"],
           ["INTERP_LAPSE_PSI2"],["INTERP_LAPSE_OVER_PSI6"],
           ["MAXNUMINTERP"]]

# Define the indices in the GRMHD_VARS.h header file
# Start with the variables used in the face value reconstructions
comment = "/* Interpolation variables */\n"
filename = "INTERP_VARS.h"
IGMout.generate_variable_definition_file(gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/INTERP_VARS.h


<a id='smallb_and_conservs_vars'></a>

## Step 1.f: The `SMALLB_VARS.h` and  `CONSERV_VARS.h` files \[Back to [top](#toc)\]
$$\label{smallb_and_conservs_vars}$$

We now set up the `SMALLB_VARS.h` and `CONSERV_VARS.h` files to substitute the following lines in the old version of the `IllinoisGRMHD_headers.h` file:

```c
// Again, the order here MATTERS, since we assume in the code that, e.g., smallb[0]=b^t, smallb[3]=b^z, etc.
static const int SMALLBT=0,SMALLBX=1,SMALLBY=2,SMALLBZ=3,SMALLB2=4,NUMVARS_SMALLB=5;

// Again, the order here MATTERS, since we assume in the code that, CONSERV[STILDEX+1] = \tilde{S}_y
static const int RHOSTAR=0,STILDEX=1,STILDEY=2,STILDEZ=3,TAUENERGY=4,NUM_CONSERVS=5;
```

In [7]:
# b^{\mu} quantities
gfslist = [["SMALLBT"],["SMALLBX"],["SMALLBY"],["SMALLBZ"],["SMALLB2"],["NUMVARS_SMALLB"]]

# Set up the b^{\mu} string
comment = "/* smallb (b^{\mu}) variables */\n"
filename = "SMALLB_VARS.h"
IGMout.generate_variable_definition_file(gfslist,filename,comment=comment)

# Conservative quantities
gfslist = [["RHOSTAR"],["STILDEX"],["STILDEY"],["STILDEZ"],["TAUENERGY"],["NUM_CONSERVS"]]

# Set up the b^{\mu} string
comment = "/* Interpolation variables */\n"
filename = "CONSERV_VARS.h"
IGMout.generate_variable_definition_file(gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/SMALLB_VARS.h
Just generated the file: ../src/NRPy_generated_headers/CONSERV_VARS.h


<a id='tmunu_vars'></a>

## Step 1.g: The `TMUNU_VARS.h` files \[Back to [top](#toc)\]
$$\label{tmunu_vars}$$

The `TMUNU_VARS.h` files sets the position of $T^{\mu\nu}$ components to be access within all $T^{\mu\nu}$ arrays, regardless if it is contravariant, covariant, or "mixed" (i.e. $T^{\mu}_{\ \ \nu}$)

In [8]:
# Then, the right/left values of the 3-velocity
gfslist = [["TMUNU_TT"]]
for i in range(DIM):
    gfslist.append(["TMUNU_T"+chr(ord('X')+i)])

for i in range(DIM):
    for j in range(i,DIM):
        gfslist.append(["TMUNU_"+chr(ord('X')+i)+chr(ord('X')+j)])

# Set up comments
comment = "/* Define TMUNU variables (valid for all variants) */\n"
# Set up output file name
filename = "TMUNU_VARS.h"
# Generate variable definition file
IGMout.generate_variable_definition_file(gfslist,filename,comment=comment)

Just generated the file: ../src/NRPy_generated_headers/TMUNU_VARS.h


<a id='igm_headers__h'></a>

# Step 2: The `IllinoisGRMHD_headers.h` file \[Back to [top](#toc)\]
$$\label{igm_headers__h}$$

We will now document the `IllinoisGRMHD_headers.h` file, even though this is probably one of the most straightforward files in `IllinoisGRMHD`. We start by going over the following three macros:

```c
#define MIN(a,b) ( ((a) < (b)) ? (a) : (b) )
#define MAX(a,b) ( ((a) > (b)) ? (a) : (b) )
#define SQR(x) ((x) * (x))
```

which are basically used for our convenience. Using `SQR(x)`, for example, is faster than using `pow(x,2)` and completely equivalent to doing `x*x`, but is very convenient when dealing with large expressions. The `MIN` and `MAX` return the minimum and maximum value of two input numbers, respectively.

We have also a few more definitions

```c
#define ONE_OVER_SQRT_4PI 0.282094791773878143474039725780

#define VERR_DEF_PARAMS __LINE__, __FILE__, CCTK_THORNSTRING

#define TINYDOUBLE 1e-100
```

which are there for convenience. The `ONE_OVER_SQRT_4PI` macro precomputes $\frac{1}{\sqrt{4\pi}}$ to many significant digits. The `TINYDOUBLE` macro is used to set a nonzero, yet extremely small value to avoid division by zero in a few computations.

Next we include all files we have generated in [Step 1](#vars_headers).

Then we set the ghostzones struct,

```c
struct gf_and_gz_struct {
  CCTK_REAL *gf;
  int gz_lo[4],gz_hi[4];
};
```

which is extremely useful to set the ghostzones of each gridfunction and keep track of it in different functions. Note that so that we keep things consistent, we update the values of `gz_lo` and `gz_hi` appropriately within each function.

Then comes the equation of state (EOS) struct,

```c
struct eos_struct {
  int neos;
  CCTK_REAL rho_ppoly_tab[MAX_EOS_PARAMS-1];
  CCTK_REAL eps_integ_const[MAX_EOS_PARAMS],K_ppoly_tab[MAX_EOS_PARAMS],Gamma_ppoly_tab[MAX_EOS_PARAMS];
};
```

which sets all EOS parameters. This struct is currently specialized to simple and piecewise polytropic EOSs.

Next comes the stats struct,

```c
struct output_stats {
  int font_fixed,vel_limited,failure_checker,rho_star_fix_applied;
  long n_iter;
};
```

which is used to identify when particularly fixes are applied in the conservative-to-primitive routine.

Then we define the [Kronecker delta](https://en.wikipedia.org/wiki/Kronecker_delta)

```c
const int kronecker_delta[4][3] = { { 0,0,0 },
                                    { 1,0,0 },
                                    { 0,1,0 },
                                    { 0,0,1 } };
```

Finally we set the prototypes of the `IllinoisGRMHD_enforce_limits_on_primitives_and_recompute_conservs()` ([**tutorial**](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb)), `IllinoisGRMHD_convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij()` ([**tutorial**](Tutorial-IllinoisGRMHD__convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.ipynb)), and `IllinoisGRMHD_set_symmetry_gzs_staggered()` ([**tutorial**](Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.ipynb)) functions.

In [9]:
%%writefile $outdir/IllinoisGRMHD_headers.h
// To safeguard against double-including this header file:
#ifndef ILLINOISGRMHD_HEADERS_H_
#define ILLINOISGRMHD_HEADERS_H_

#define MIN(a,b) ( ((a) < (b)) ? (a) : (b) )
#define MAX(a,b) ( ((a) > (b)) ? (a) : (b) )
#define SQR(x) ((x) * (x))
#define ONE_OVER_SQRT_4PI 0.282094791773878143474039725780

#define VERR_DEF_PARAMS __LINE__, __FILE__, CCTK_THORNSTRING

#define TINYDOUBLE 1e-100

#include "NRPy_generated_headers/ADM_3METRIC_VARS.h"

#include "NRPy_generated_headers/CONF_METRIC_FACEVALS_VARS.h"

#include "NRPy_generated_headers/GRMHD_VARS.h"

#include "NRPy_generated_headers/INTERP_VARS.h"

#include "NRPy_generated_headers/SMALLB_VARS.h"

#include "NRPy_generated_headers/CONSERV_VARS.h"

#include "NRPy_generated_headers/TMUNU_VARS.h"

// Keeping track of ghostzones between routines is a nightmare, so
//   we instead attach ghostzone info to each gridfunction and set
//   the ghostzone information correctly within each routine.
struct gf_and_gz_struct {
  CCTK_REAL *gf;
  int gz_lo[4],gz_hi[4];
};

#define MAX_EOS_PARAMS 10
struct eos_struct {
  int neos;
  CCTK_REAL rho_ppoly_tab[MAX_EOS_PARAMS-1];
  CCTK_REAL eps_integ_const[MAX_EOS_PARAMS],K_ppoly_tab[MAX_EOS_PARAMS],Gamma_ppoly_tab[MAX_EOS_PARAMS];
};

struct output_stats {
  int font_fixed,vel_limited,failure_checker,rho_star_fix_applied;
  long n_iter;
};


// FIXME: For cosmetic purposes, we might want to make everything either zero-offset or one-offset, instead of a mixture.
const int kronecker_delta[4][3] = { { 0,0,0 },
                                    { 1,0,0 },
                                    { 0,1,0 },
                                    { 0,0,1 } };

/* PUBLIC FUNCTIONS, USED OUTSIDE IllinoisGRMHD AS WELL */
void IllinoisGRMHD_enforce_limits_on_primitives_and_recompute_conservs(const int already_computed_physical_metric_and_inverse,CCTK_REAL *U,struct output_stats &stats,eos_struct &eos,
                                                                       CCTK_REAL *METRIC,CCTK_REAL g4dn[4][4],CCTK_REAL g4up[4][4], CCTK_REAL *TUPMUNU,CCTK_REAL *TDNMUNU,CCTK_REAL *CONSERVS);

void IllinoisGRMHD_convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij
(const cGH *cctkGH,const int *cctk_lsh,
 CCTK_REAL *gxx,CCTK_REAL *gxy,CCTK_REAL *gxz,CCTK_REAL *gyy,CCTK_REAL *gyz,CCTK_REAL *gzz,CCTK_REAL *alp,
 CCTK_REAL *gtxx,CCTK_REAL *gtxy,CCTK_REAL *gtxz,CCTK_REAL *gtyy,CCTK_REAL *gtyz,CCTK_REAL *gtzz,
 CCTK_REAL *gtupxx,CCTK_REAL *gtupxy,CCTK_REAL *gtupxz,CCTK_REAL *gtupyy,CCTK_REAL *gtupyz,CCTK_REAL *gtupzz,
 CCTK_REAL *phi,CCTK_REAL *psi,CCTK_REAL *lapm1);

void IllinoisGRMHD_set_symmetry_gzs_staggered(const cGH *cctkGH, const int *cctk_lsh,CCTK_REAL *X,CCTK_REAL *Y,CCTK_REAL *Z,  CCTK_REAL *gridfunc,
                                              CCTK_REAL *gridfunc_syms,int stagger_x,int stagger_y,int stagger_z);

#include "IllinoisGRMHD_EoS_lowlevel_functs.C"
#endif // ILLINOISGRMHD_HEADERS_H



Overwriting ../src/IllinoisGRMHD_headers.h


<a id='code_validation'></a>

# Step 3: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [10]:
# # Verify if the code generated by this tutorial module
# # matches the original IllinoisGRMHD source code

# # First download the original IllinoisGRMHD source code
# import urllib
# from os import path

# original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/IllinoisGRMHD_headers.h"
# original_IGM_file_name = "IllinoisGRMHD_headers-original.h"
# original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# # Then download the original IllinoisGRMHD source code
# # We try it here in a couple of ways in an attempt to keep
# # the code more portable
# try:
#     original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
#     # Write down the file the original IllinoisGRMHD source code
#     with open(original_IGM_file_path,"w") as file:
#         file.write(original_IGM_file_code)
# except:
#     try:
#         original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
#         # Write down the file the original IllinoisGRMHD source code
#         with open(original_IGM_file_path,"w") as file:
#             file.write(original_IGM_file_code)
#     except:
#         # If all else fails, hope wget does the job
#         !wget -O $original_IGM_file_path $original_IGM_file_url

# # Perform validation
# Validation__IllinoisGRMHD_headers__h  = !diff $original_IGM_file_path $outfile_path__IllinoisGRMHD_headers__h

# if Validation__IllinoisGRMHD_headers__h == []:
#     # If the validation passes, we do not need to store the original IGM source code file
#     !rm $original_IGM_file_path
#     print("Validation test for IllinoisGRMHD_headers.h: PASSED!")
# else:
#     # If the validation fails, we keep the original IGM source code file
#     print("Validation test for IllinoisGRMHD_headers.h: FAILED!")
#     # We also print out the difference between the code generated
#     # in this tutorial module and the original IGM source code
#     print("Diff:")
#     for diff_line in Validation__IllinoisGRMHD_headers__h:
#         print(diff_line)

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__IllinoisGRMHD_headers.pdf](Tutorial-IllinoisGRMHD__IllinoisGRMHD_headers.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [11]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__IllinoisGRMHD_headers.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__IllinoisGRMHD_headers.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__IllinoisGRMHD_headers.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__IllinoisGRMHD_headers.tex
!rm -f Tut*.out Tut*.aux Tut*.log